In [1]:

import keras
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 
from keras.models import Sequential 
from keras import optimizers
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Conv2D, Dense 
from keras import applications 
from keras.utils.np_utils import to_categorical 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
from keras.applications.vgg16 import preprocess_input
import pandas as pd
import numpy as np 
import itertools
%matplotlib inline
import math 
import datetime
import time


Using TensorFlow backend.


In [ ]:
# train_path='./train/'
# test_path='./test/'


# train_img=[]
# for i in range(len(train)):

#     temp_img=image.load_img(train_path+train['filename'][i],target_size=(224,224))

#     temp_img=image.img_to_array(temp_img)

#     train_img.append(temp_img)

# #converting train images to array and applying mean subtraction processing

# train_img=np.array(train_img)
# train_img=preprocess_input(train_img)



# test_img=[]
# for i in range(len(test)):

#     temp_img=image.load_img(test_path+test['filename'][i],target_size=(224,224))

#     temp_img=image.img_to_array(temp_img)

#     test_img.append(temp_img)

# test_img=np.array(test_img)
# test_img=preprocess_input(test_img)

In [2]:


#Create a bottleneck file
top_model_weights_path = 'bottleneck_fc_model_101.h5'

traindf = pd.read_csv('./train.csv',dtype="str")
testdf=pd.read_csv('./test.csv',dtype="str")



In [3]:
def append_ext(fn):
    return fn+".png"

traindf["id"]=traindf["id"].apply(append_ext)
testdf["id"]=testdf["id"].apply(append_ext)

In [4]:
train_data = ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess_input).flow_from_dataframe(dataframe=traindf, directory="./train/", x_col="id",y_col="label", batch_size=128,target_size=(32,32))
# test_data = ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess_input).flow_from_dataframe(dataframe=testdf, directory="./test/", x_col="id",y_col= None, batch_size=32,target_size=(32,32),class_mode=None)


Found 60000 validated image filenames belonging to 10 classes.
Found 10000 validated image filenames.


In [4]:
img_width, img_height = 224,224
vgg16 = applications.VGG16(include_top=False, weights='imagenet',input_shape=(img_width, img_height,3))


In [5]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [7]:
batch_size=32
train_data = ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess_input).flow_from_dataframe(dataframe=traindf, directory="./train/",
                                          x_col="id",y_col="label", batch_size=32,target_size=(img_width, img_height),
                                         class_mode=None,shuffle=False)

nb_train_samples = len(train_data.filenames)
print(nb_train_samples)
 
predict_size_train = int(math.ceil(nb_train_samples / batch_size))
print(predict_size_train)

Found 60000 validated image filenames.
60000
1875


In [8]:
start = datetime.datetime.now()

 

bottleneck_features_train= vgg16.predict_generator(train_data, predict_size_train) 
 
np.save('bottleneck_features_train_101.npy', bottleneck_features_train)
end= datetime.datetime.now()
elapsed= end-start
print ('Time:', elapsed)

Time: 2:04:11.041172


In [9]:
train_data_top = ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess_input).flow_from_dataframe(dataframe=traindf, directory="./train/",
                                         x_col="id",y_col="label", batch_size=batch_size,target_size=(img_width, img_height),
                                        class_mode='categorical',shuffle=False)
    
nb_train_samples = len(train_data_top.filenames) 
num_classes = len(train_data_top.class_indices) 
 
# load the bottleneck features saved earlier 
train_data = np.load('bottleneck_features_train_101.npy') 

# get the class labels for the training data, in the original order 
train_labels = train_data_top.classes 
 
# get the class labels for the training data, in the original order 
train_labels = to_categorical(train_labels, num_classes=num_classes)
 

Found 60000 validated image filenames belonging to 10 classes.


In [13]:
from keras.layers import BatchNormalization
from keras.layers import Activation, Dense,GlobalAveragePooling2D,LeakyReLU
from keras.layers import Dropout
from keras.models import Model

start = datetime.datetime.now()
model = Sequential()

# model.add(Conv2D(input_shape=train_data.shape[1:],filters=600,kernel_size=(3,3),padding="same"))
# model.add(BatchNormalization())          
# model.add(Activation('relu'))
# model.add(Conv2D(filters=600,kernel_size=(3,3),padding="same"))
# model.add(BatchNormalization())          
# model.add(Activation('relu'))
# model.add(Conv2D(filters=600,kernel_size=(3,3),padding="same"))
# model.add(BatchNormalization())          
# model.add(Activation('relu'))


model.add(Flatten())
model.add(Dense(250))
model.add(LeakyReLU(alpha=0.3)) 
model.add(Dropout(0.5)) 
model.add(Dense(100)) 
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(0.3)) 
model.add(Dense(num_classes, activation='softmax'))





model.compile(loss='categorical_crossentropy',
   optimizer=optimizers.Adam(lr=1e-4),
   metrics=['acc'])
history = model.fit(train_data, train_labels, 
   epochs=20,
   batch_size=32)

model.save_weights(top_model_weights_path)

end= datetime.datetime.now()
elapsed= end-start
print ('Time:', elapsed)

Epoch 1/20
60000/60000 [==============================] - 193s 3ms/step - loss: 0.5074 - acc: 0.8219
Epoch 2/20
60000/60000 [==============================] - 192s 3ms/step - loss: 0.3573 - acc: 0.8728
Epoch 3/20
60000/60000 [==============================] - 192s 3ms/step - loss: 0.3165 - acc: 0.8875
Epoch 4/20
60000/60000 [==============================] - 191s 3ms/step - loss: 0.2900 - acc: 0.8977
Epoch 5/20
60000/60000 [==============================] - 191s 3ms/step - loss: 0.2723 - acc: 0.9020
Epoch 6/20
60000/60000 [==============================] - 191s 3ms/step - loss: 0.2596 - acc: 0.9074
Epoch 7/20
60000/60000 [==============================] - 190s 3ms/step - loss: 0.2508 - acc: 0.9093
Epoch 8/20
60000/60000 [==============================] - 190s 3ms/step - loss: 0.2355 - acc: 0.9144
Epoch 9/20
60000/60000 [==============================] - 190s 3ms/step - loss: 0.2276 - acc: 0.9179
Epoch 10/20
60000/60000 [==============================] - 189s 3ms/step - loss: 0.2202 - a

In [14]:
from keras.models import model_from_json

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_101.h5")
print("Saved model to disk")

Saved model to disk


In [34]:
test_data = ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess_input).flow_from_dataframe(dataframe=testdf, directory="./test/", x_col="id",y_col= None, batch_size=128,target_size=(img_width, img_height),class_mode=None,shuffle=False)

nb_test_samples = len(test_data.filenames) 
predict_size_test = int(math.ceil(nb_test_samples / 128))

start = datetime.datetime.now()


bottleneck_features_test= vgg16.predict_generator(test_data, predict_size_test) 
 
np.save('bottleneck_features_test.npy', bottleneck_features_test)

end= datetime.datetime.now()
elapsed= end-start
print ('Time:', elapsed)

Found 10000 validated image filenames.
Time: 0:00:41.890111


In [37]:
#test data
test_data_top = ImageDataGenerator(
    rescale=1./255,preprocessing_function=preprocess_input).flow_from_dataframe(dataframe=testdf,
    directory="./test/", x_col="id",y_col= None, batch_size=128,target_size=(img_width, img_height),
                                                                                class_mode=None,shuffle=False)
 
nb_test_samples = len(test_data_top.filenames) 
 
# load the bottleneck features saved earlier 
test_data = np.load('bottleneck_features_test.npy') 
 
# # get the class labels for the training data, in the original order 
# test_labels = test_data_top.classes 
 
# # convert the training labels to categorical vectors 
# test_labels = to_categorical(test_labels, num_classes=num_classes)

Found 10000 validated image filenames.


In [39]:
# nb_test_samples = len(test_data.filenames) 
# predict_size_test = int(math.ceil(nb_test_samples / 128))

def test_single_image(path):
  classs = ['0','1','2','3','4','5','6','7','8','9']
  images = read_image(path)
  time.sleep(.5)
  bt_prediction = vgg16.predict(images) 
  preds = model.predict_proba(bt_prediction)
  for idx, animal, x in zip(range(0,10), animals , preds[0]):
   print(“ID: {}, Label: {} {}%”.format(idx, animal, round(x*100,2) ))
  print(‘Final Decision:’)
  time.sleep(.5)
  for x in range(3):
   print(‘.’*(x+1))
   time.sleep(.2)
  class_predicted = model.predict_classes(bt_prediction)
  class_dictionary = _top.class_indices 
  inv_map = {v: k for k, v in class_dictionary.items()} 
  print(“ID: {}, Label: {}”.format(class_predicted[0],  inv_map[class_predicted[0]])) 
  return load_img(path)


path = './test/'
test_single_image(path)

ValueError: Error when checking input: expected conv2d_14_input to have shape (1, 1, 512) but got array with shape (32, 32, 3)

In [7]:
test_generator = ImageDataGenerator(rescale=1./255.).flow_from_directory(
    directory="./test/",
    target_size=(32,32),
    batch_size=128,
    class_mode=None,
    shuffle=False
    
)

Found 0 images belonging to 0 classes.


In [13]:
from keras.models import model_from_json

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
